In [309]:
import numpy as np
import cv2 as cv
from matplotlib import pyplot as plt
from pathlib import Path
import math
from source.side import Side
from source.dice import Dice

# from ipynb.fs.full.functions import isCircle
from source.isCircle import isCircle
from source.clustering import clustering, distSquare
# good
"2024-06-08_23-37-01_original" # 6,6,2
"2024-06-09_01-02-59_original" # 3,6,5
"2024-05-07_16-07-07_original"
"2024-06-09_07-06-07_original"
'2024-06-09_07-06-07_original'
"2024-06-09_05-36-23_original"
"2024-06-09_05-09-08_original"
"2024-06-08_22-32-54_original"
"2024-06-08_22-48-25_original"
"2024-06-09_07-04-58_original"
# bad
"2024-05-08_10-09-13_original"
"2024-05-09_05-55-43_original"
"2024-05-09_04-54-21_original"
""
imageName = "2024-06-09_07-06-07_original"
writeToFile = True
outputFolder = f'./data/{imageName}'
diceWidth = 35

# original = cv.imread(f'./data/original/{imageName}.png')
original = cv.imread(f'./newData/{imageName}.png', cv.IMREAD_GRAYSCALE)

assert original is not None, "file could not be read"

if(writeToFile):
    Path(outputFolder).mkdir(parents=True, exist_ok=True)

In [ ]:
def crop_ring(image, crop_width, crop_height, center):
    center_y, center_x = center

    start_x = max(center_x - crop_width // 2, 0)
    start_y = max(center_y - crop_height // 2, 0)
    end_x = start_x + crop_width
    end_y = start_y + crop_height

    # Crop the image
    cropped_image = image[start_y:end_y, start_x:end_x]

    return cropped_image

ringCenter = (250, 325)
radius = 150
radiusSquared = radius ** 2
center_y, center_x = ringCenter

original = crop_ring(original, radius * 2, radius* 2, ringCenter)
height, width = original.shape

img = original.copy()
height, width = img.shape
center_y, center_x = (height //2, width//2)
Y, X = np.ogrid[:height, :width]
distance_from_center = (X - center_x) ** 2 + (Y - center_y) ** 2
mask = distance_from_center >= radiusSquared
img[mask] = 0


plt.imshow(img, cmap='gray')
plt.axis("off")

In [ ]:
edges = cv.Canny(img, 100, 100)
cv.imwrite(f'{outputFolder}/edges.png', edges)

plt.imshow(edges, cmap='gray')
plt.axis('off')

In [ ]:
onlyWhite = img.copy()

non_white_mask = onlyWhite <= 250
reverse_mask = ~non_white_mask
# Set all non-white pixels to black (0)
onlyWhite[non_white_mask] = 0
onlyWhite[reverse_mask] = 255

if(writeToFile):
    cv.imwrite(f'{outputFolder}/whitePixels.png', onlyWhite)

plt.imshow(onlyWhite, cmap='gray')
plt.axis('off')
            

In [ ]:
# [ 254, 100, 12, ...]
sectorsCounts = [0]
# [0,1,0,4,4,4,0,0,0,1...]
imgSectorMap = np.zeros((height, width), dtype=np.uint16)
# [0,1,0,0,0,0 ...]
isPixelChecked = np.zeros((height, width), dtype=np.bool)
# [[y,x, sector], ...]
sectorsByPixels = []


def findSector(startingY, startingX, number):
    global onlyWhite, sectorsByPixels, imgSectorMap, sectorsCounts
    arr = [[startingY, startingX]]
    while len(arr):
        poped = arr.pop()
        y = poped[0]
        x = poped[1]

        if y == height:
            continue
        if x == width:
            continue
        if x < 0:
            continue
        if y < 0:
            continue

        if isPixelChecked[y, x] == 0:
            if onlyWhite[y, x] == 255:
                isPixelChecked[y, x] = 1
                imgSectorMap[y, x] = number
                sectorsByPixels.append([y, x, number])
                sectorsCounts[number] = sectorsCounts[number] + 1

                arr.append([y, x + 1, number])
                arr.append([y, x - 1, number])
                arr.append([y + 1, x, number])
                arr.append([y - 1, x, number])


for y in range(0, height):
    for x in range(0, width):
        if onlyWhite[y, x] >= 254 and imgSectorMap[y, x] == 0:
            sectorsCounts.append(0)
            findSector(y, x, len(sectorsCounts) - 1)

validDots = []
validPixels = []

# y,x, height, width, cy, cx
sectorDimensions = []

for index, sector in enumerate(sectorsCounts):
    xmin = 999999
    ymin = 999999
    xmax = -1
    ymax = -1
    sectorDimensions.append([0, 0, 0, 0, 0, 0])
    if sector == 0:
        continue

    if sector <= 15 or sector >= 100:
        for x_y_sector_arr in sectorsByPixels:
            s = x_y_sector_arr[2]
            if s == index:
                y = x_y_sector_arr[0]
                x = x_y_sector_arr[1]
                onlyWhite[y, x] = 0
    else:
        validDots.append([])
        for x_y_sector_arr in sectorsByPixels:
            s = x_y_sector_arr[2]
            if s == index:
                y = x_y_sector_arr[0]
                x = x_y_sector_arr[1]
                validPixels.append([y, x])
                validDots[len(validDots) - 1].append([y, x])
                if ymin > y:
                    ymin = y
                if ymax < y:
                    ymax = y
                if xmin > x:
                    xmin = x
                if xmax < x:
                    xmax = x
        w = xmax - xmin + 1
        h = ymax - ymin + 1
        cx = xmin + w / 2
        cy = ymin + h / 2
        
        if w > 20 or h > 20:
            for x_y_sector_arr in sectorsByPixels:
                s = x_y_sector_arr[2]
                if s == index:
                    y = x_y_sector_arr[0]
                    x = x_y_sector_arr[1]
                    onlyWhite[y, x] = 0
        else:
            sectorDimensions[index] = [ymin, xmin, h, w, cy, cx]

def filterEmpty(x):
    if x[2] == 0:
        return False
    if x[3] == 0:
        return False
    return True

sectorDimensions = list(filter(filterEmpty, sectorDimensions))

if writeToFile:
    cv.imwrite(f"{outputFolder}/dots.png", onlyWhite)

plt.imshow(onlyWhite, cmap="gray")
plt.axis("off")

In [ ]:

spaceBetweenDots = math.floor(diceWidth / 2)
sidesClusters = clustering(sectorDimensions, spaceBetweenDots, diceWidth)
diceClusters = clustering(sectorDimensions, spaceBetweenDots, diceWidth + 20)

dices: list[Dice] = list(map(lambda x: Dice(), range(len(diceClusters))))

for index, diceCluster in enumerate(diceClusters):
    for index2, sideCluster in enumerate(sidesClusters):
        check = False
        for p in diceCluster:
            for p2 in sideCluster:
                if p[0] == p2[0] and p[1] == p2[1]:
                    check = True
                    break
        if check:
            side = Side(original)
            side.dimensions = list(map(lambda x: x[2], sideCluster))
            dices[index].sides.append(side)


for dice in dices:
    if len(dice.sides) == 1:
        dice.isValid = dice.sides[0].isValid
        dice.value = dice.sides[0].value
        continue
    if len(dice.sides) > 1:
        countValid = 0

        validSides: list[Side] = []
        invalidSides: list[Side] = []
        for side in dice.sides:
            if side.isValid:
                validSides.append(side)
                continue
            else:
                invalidSides.append(side)
        if len(validSides) != 1:
            dice.isValid = False
            dice.value = -1
            continue
        validSide = validSides[0]
        check = True
        for invalidSide in invalidSides:
            centerOFImage: list[float] = [height / 2, width / 2]
            d1 = distSquare(centerOFImage, invalidSide.center)
            d2 = distSquare(centerOFImage, validSide.center)
            if d2 < d1:
                check = False

        if check:
            dice.isValid = True
            dice.value = validSide.value

print(dices[2].sides[0].dimensions)

tmpImage = original.copy()
for index, dice in enumerate(dices):
    print(len(dice.sides))
    print(list(map(lambda x: x.dimensions, dice.sides)))
    cy, cx = dice.center
    rcx = math.floor(cx)
    rcy = math.floor(cy)
    tmpImage = cv.circle(tmpImage, (rcx, rcy), 40, (0, 255, 255, 0.1))
    if dice.isValid:
        cv.putText(
            tmpImage,
            f"{dice.value}",
            (rcx, rcy),
            cv.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 0),
            2,
            cv.LINE_4,
        )
    else:
        cv.putText(
            tmpImage,
            f'{"IN"}',
            (rcx, rcy),
            cv.FONT_HERSHEY_SIMPLEX,
            1,
            (255, 255, 0),
            2,
            cv.LINE_4,
        )

if writeToFile:
    tmpImage = np.concatenate((original, tmpImage), axis=0)
    cv.imwrite(f"{outputFolder}/score.png", tmpImage)

plt.imshow(tmpImage, cmap='gray')
plt.axis("off")

In [ ]:
tmpImg = np.zeros_like(img, dtype=np.uint8)
print(len(sidesClusters))
for p in sidesClusters[1]:
    tmpImg = cv.circle(tmpImg, (math.floor(p[1]), math.floor(p[0])), 5, 255, -1)

plt.figure(figsize=(30, 5))
plt.subplot(1, 1, 1)
plt.imshow(tmpImg, cmap='gray')
plt.axis('off')
plt.subplot(1, 2, 1)
plt.imshow(onlyWhite, cmap='gray')
plt.axis('off')

In [ ]:
tmpImg = np.zeros((height, width, 3), dtype=np.uint8)
for dimension in sectorDimensions:
    y, x, h, w, cy, cx = dimension
    if w == 0:
        continue
    if h == 0:
        continue

    padding = 3
    for i in range(math.floor(y - padding), math.floor(y + h + padding + 1)):
        for j in range(math.floor(x - padding), math.floor(x + w + padding + 1)):
            tmpImg[i, j] = original[i, j].copy()


if writeToFile:
    cv.imwrite(f"{outputFolder}/dices_colored.png", tmpImg)

plt.imshow(tmpImg, cmap="gray")
plt.axis("off")

In [ ]:
tmpImg2 = cv.cvtColor(original, cv.COLOR_GRAY2RGB)

for index, dimension in enumerate(sectorDimensions):
    dimension = sectorDimensions[index]
    # if(index != 8):
    #     continue
    check = isCircle(dimension, original)
    print(check, dimension)
    color = (0, 0, 255)
    if check:
        color = (0, 255, 0)

    tmpImg2 = cv.circle(
        tmpImg2, (math.floor(dimension[5]), math.floor(dimension[4])), 5, color, -1
    )

if writeToFile:
    cv.imwrite(f"{outputFolder}/valid_dots.png", tmpImg2)
    
    
plt.imshow(cv.cvtColor(tmpImg2, cv.COLOR_BGR2RGB))
plt.axis("off")

In [ ]:
tmpImg2 = cv.cvtColor(original, cv.COLOR_GRAY2RGB)

length = len(sectorDimensions)

all = []
for i in range(0, length):
    for j in range(0, length):
        dimension1 = sectorDimensions[i]
        dimension2 = sectorDimensions[j]
        y, x, h, w, cy1, cx1 = dimension1
        y, x, h, w, cy2, cx2 = dimension2
        all.append(np.array(((cy1 + cy2) // 2, (cx1 + cx2) // 2), dtype=np.uint16))

i = 0
while i < len(all):
    check = True
    for dimension in sectorDimensions:
        y, x, h, w, cy, cx = dimension
        dist = distSquare(all[i], (cy, cx))
        if dist < 40 * 40:
            check = False
            break

    if check:
        all.pop(i)
    else:
        i += 1


def findSector(point, isPixelChecked, onlyWhite):
    arr = [point]
    pixels = []
    while len(arr):
        y, x = arr.pop()

        if y >= height:
            continue
        if x >= width:
            continue
        if x < 0:
            continue
        if y < 0:
            continue

        if isPixelChecked[y, x] == 0:
            if onlyWhite[y, x] == 255:
                isPixelChecked[y, x] = 1
                pixels.append((y, x))
                arr.append((y, x + 1))
                arr.append((y, x - 1))
                arr.append((y + 1, x))
                arr.append((y - 1, x))

    cx = 0
    cy = 0
    for p in pixels:
        cx += p[1]
        cy += p[0]

    if len(pixels) == 0:
        return (0, 0)

    cx /= len(pixels)
    cy /= len(pixels)
    return (cy, cx)



def find_angle_between_points(p1, p2):
    y1, x1 = p1
    y2, x2 = p2
    dx = x2 - x1
    dy = y2 - y1

    angle_radians = math.atan2(dy, dx)

    angle_degrees = math.degrees(angle_radians)
    
    return angle_degrees

def are_opposite_angles(angle1, angle2, tolerance=20):
    angle1 = angle1 % 360
    angle2 = angle2 % 360
    
    difference = abs(angle1 - angle2)
    
    if difference > 180:
        difference = 360 - difference
    
    return 180 - tolerance <= difference <= 180 + tolerance

def checkDot(point, onlyWhite):
    y, x = point
    r = 15
    isPixelChecked = np.zeros((height, width), dtype=np.bool)

    sectors = []
    for i in range(y - r, y + r + 1):
        for j in range(x - r, x + r + 1):
            if isPixelChecked[i, j] == 0:
                s = findSector((i, j), isPixelChecked, onlyWhite)
                if isPixelChecked[i, j] == 1:
                    sectors.append(s)

    distances = []
    
    if(len(sectors) == 0):
        return -1

    if len(sectors) % 2 == 0:
        if onlyWhite[y, x] != 0:
            return -1
    else:
        if onlyWhite[y, x] != 255:
            return -1

    for p in sectors:
        distances.append((math.sqrt(distSquare(p, point)), p))

    distances = sorted(distances, key=lambda x: x[0])
    d1 = distances[0]
    count = 0
    sectored = [[]]

    for d2 in distances:
        if abs(d1[0] - d2[0]) < 2:
            sectored[count].append(d2)
            continue
        sectored.append([d2])
        count = 1
        d1 = d2

    countDistances = []
    for s in sectored:
        countDistances.append(len(s))

    
    if (
        countDistances != [1]
        and countDistances != [2]
        and countDistances != [1, 2]
        and countDistances != [4]
        and countDistances != [1, 4]
        and countDistances != [2, 4]
    ):
        return -1


    for s in sectored:
        if(len(s)<=1):
            continue
        angles = [find_angle_between_points(point,item[1])%360 for item in s]
        countChecks = 0
        for index1, angle1 in enumerate(angles):
            check = 0
            for index2, angle2 in enumerate(angles):
                if(index2<= index1):
                    continue
                if are_opposite_angles(angle1, angle2):
                    check = 1
                    
            countChecks += check
        if(countChecks != len(s)/2):
            return -1
                    
    if countDistances == [1]:
        return 1
    if countDistances == [2]:
        return 2
    if countDistances == [1, 2]:
        return 3
    if countDistances == [4]:
        return 4
    if countDistances == [1, 4]:
        return 5
    if countDistances == [2, 4]:
        return 6
    return -1


i = 0
while i < len(all):
    j = i + 1
    while j < len(all):
        p1 = all[i]
        p2 = all[j]
        if distSquare(p1, p2) < 3:
            all.pop(j)
        else:
            j += 1
    i += 1

valid = []
for index, point in enumerate(all):
    # if(index != 17):
    #     continue
    dotValue = checkDot(point, onlyWhite)
    if dotValue != -1:
        valid.append((dotValue, index, point))
    # color = (0, 0, 255)
    # y, x = point
    # tmpImg2 = cv.circle(tmpImg2, (x, y), 1, color, -1)


i = 0
isInvalid = False
while i < len(valid):
    v1, _, p1 = valid[i]
    j = i + 1
    while j < len(valid):
        v2, _, p2 = valid[j]
        if(distSquare(p1,p2) < 20*20):
            if(v1<v2):
                valid.pop(i)
                i-=1
                break
            if(v1 == v2):
                isInvalid = True
                break
            if(v1>v2):
                valid.pop(j)
                j-=1
        j+=1
    i += 1

for index, v in enumerate(valid):
        v, i, p = valid[index]
        color = (0, 0, 255)
        y, x = p
        print(v,i)
        tmpImg2 = cv.circle(tmpImg2, (x, y), 1, color, -1)
        

plt.imshow(cv.cvtColor(tmpImg2, cv.COLOR_BGR2RGB))
plt.axis("off")

In [ ]:
contours, _ = cv.findContours(onlyWhite, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)
lens = []
image = cv.cvtColor(original, cv.COLOR_GRAY2RGB)
black = np.zeros_like(original)
print(black.shape)
for contour in contours:
    x, y, w, h = cv.boundingRect(contour)
    lens.append(len(contour))
    cv.drawContours(black, [contour], -1, 255, thickness=cv.FILLED)

print(lens)

difference = cv.absdiff(onlyWhite, black)
plt.imshow(black, cmap="gray")
plt.axis("off")


In [ ]:
# 139  55
diceW = 20
start_y = 139 - diceW
end_y = 139 + diceW
start_x = 55 -diceW
end_x = 55 + diceW

cropped_image = image[start_y:end_y, start_x:end_x]
plt.imshow(cropped_image, cmap='gray')


In [ ]:
tmpImg2 = cv.cvtColor(original, cv.COLOR_GRAY2RGB)

i = 0
isInvalid = False
while i < len(valid):
    v1, _, p1 = valid[i]
    j = i + 1
    while j < len(valid):
        v2, _, p2 = valid[j]
        if distSquare(p1, p2) < 20 * 20:
            if v1 < v2:
                valid.pop(i)
                i -= 1
                break
            if v1 == v2:
                isInvalid = True
                break
            if v1 > v2:
                valid.pop(j)
                j -= 1
        j += 1
    i += 1

tmpValid = valid.copy()
tmpValid = sorted(tmpValid, key=lambda x: x[2][0])

tmpSectors = [[sector, False] for sector in sectorDimensions]

print(tmpSectors)
print(tmpValid)
validFiltered = []
popFront = True
while len(tmpValid):
    if popFront:
        v, i, p = tmpValid.pop(0)
    else:
        v, i, p = tmpValid.pop(0)
    popFront = not popFront

    def dist_square_filter(sector):
        y, x, h, w, cy, cx = sector[0]
        return distSquare(p, (cy, cx)) < 30 * 30

    indexes = []
    for i, s in enumerate(tmpSectors):
        if dist_square_filter(s):
            indexes.append(i)

    nearSectors = list(filter(dist_square_filter, tmpSectors))
    boolSectors = list(map(lambda x: x[1], nearSectors))
    isChecked = any(boolSectors)

    if not isChecked:
        for i in indexes:
            tmpSectors[i][1] = True
        validFiltered.append((v, i, p))

    print(boolSectors)
    print(isChecked)

    print(nearSectors)
valid = validFiltered


for index, v in enumerate(valid):
    v, i, p = valid[index]
    color = (0, 0, 255)
    y, x = p
    print(v, i)
    tmpImg2 = cv.circle(tmpImg2, (x, y), 1, color, -1)

result = list(
    map(
        lambda v: f"{v[0]}",
        valid,
    )
)


plt.imshow(cv.cvtColor(tmpImg2, cv.COLOR_BGR2RGB))
plt.axis("off")